In [1]:
import numpy as np
from pySWATPlus.TxtinoutReader import TxtinoutReader
from pySWATPlus.SWATProblem import SWATProblem, minimize_pymoo
import random

from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.termination import get_termination
from pymoo.util.normalization import denormalize

In [2]:
"""
The user defined function must receive only one parameter, which is a dictionary taht can have any structure, but must have
one key (with any user-specified name, in our case is 'calibration_params') that contains the calibration parameters. It must be in the following format: 
{filename: (id_col, [(id, col, value)])}
"""

def function_to_minimize(dict_of_params):

    calibration_params = dict_of_params['calibration_params']
    path_to_txtinout = dict_of_params['path_to_txtinout']

    #read path to txtinout
    reader = TxtinoutReader(path_to_txtinout)
    tmp_path = reader.copy_swat(dir = None) #Copy to temporary directory
    reader = TxtinoutReader(tmp_path)

    #run swat with calibration_params, which must be in format  {filename: (id_col, [(id, col, value)])}
    txt_in_out_result = reader.run_swat(calibration_params, show_output=False)
        
    #read path_to_txtinout
    result_reader = TxtinoutReader(txt_in_out_result)

    """
    The following steps should be: read the results, gather the observations, calculate the error based on the observations and the simulation, and finally, return the error
    """

    #Return in format Tuple[int, Dict[str, str]]
    return (random.random(), {'test_calibration': result_reader.root_folder})   #We return any random error

In [ ]:
txtinout_folder = 'path/to/txtinout'

swat_problem = SWATProblem(params = {'plants.plt': ('name', [('bana', 'bm_e', 40, 50), ('bana', 'harv_idx', 0.4, 0.5)])},
                           function_to_evaluate=function_to_minimize,
                           param_arg_name='calibration_params',
                           n_workers=4,
                           parallelization = 'threads',
                           debug=True,
                           path_to_txtinout=txtinout_folder)